<h1><center>Universidade Federal de Minas Gerais</center></h1>
<h3><center>Departamento de Ciência da Computação</center></h3>
<h3><center>Introdução a Banco de Dados -  Trabalho Prático #1</center></h3>


# 1. Introdução

### 1.1. Contexto

Nesse trabalho prático utilizaremos o banco de dados representado pelo schema abaixo:

![Modelo_BD.png](ibd-main/data/schema.png)

### 1.2. Entrega




1.   Você deve completar esse notebook, executar todas as células e submetê-lo no **Minha UFMG**.
2.   Você pode entregar até o dia 23/01/2022 às 23:59.
3. O trabalho é individual e vale 15 pontos.






# 2. Preparação do Ambiente

In [1]:
# importando os packages necessários
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8a411de3574b5e2e984f18eda541052ae80acd6c9e73a5b835342ed8e1de5aa1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
import wget

In [4]:
wget.download("https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql")

'despesas_publicas_tp1.sql'

In [5]:
# download do dump do banco de  dados:
!wget https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql

--2022-01-22 12:15:44--  https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277046 (271K) [text/plain]
Saving to: ‘despesas_publicas_tp1.sql.1’

despesas_publicas_t 100%[===================>] 270.55K  --.-KB/s    in 0.03s   

2022-01-22 12:15:44 (8.79 MB/s) - ‘despesas_publicas_tp1.sql.1’ saved [277046/277046]



### 2.1. Criação da conexão com o banco
* No momento da conexão, é informado o nome do arquivo onde será montado o banco de dados. No nosso caso, este arquivo não precisa ser pré-existente, porque iremos carregar as informações de um dump;
* Se este comando for executado com um arquivo que já existe e possui as mesmas tabelas que serão criadas, ocorrerá um erro no momento da execução do script mais à frente.

In [6]:
conn = sqlite3.connect('despesas.db')
cursor = conn.cursor()

### 2.2. Execução do script para inicializar e popular o banco de dados

* Criada a conexão, iremos utilizar o arquivo .sql disponibilizado para recuperar o schema e os dados do banco

In [7]:
f = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
sql = f.read()
cursor.executescript(sql)

### 2.3. Realizando consultas no banco de dados

* Criada a conexão e realizada a carga dos dados no banco, pode ser executada qualquer consulta SQL;
* O SQLite3 permite que o retorno dos dados seja feito em vários formatos, entretanto, utilizaremos o retorno em um dataframe do Pandas.



**CONSULTA EXEMPLO:** todos os órgãos ordenados pela quantidade de orgãos subordinados de forma decrescente.

In [8]:
df = pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,26000,Ministério da Educação,104
1,30000,Ministério da Justiça e Segurança Pública,6
2,22000,"Ministério da Agricultura, Pecuária e Abastec",5
3,24000,"Ministério da Ciência, Tecnologia, Inovações",4
4,52000,Ministério da Defesa,4
5,53000,Ministério da Integração Nacional,4
6,25000,Ministério da Fazenda,3
7,36000,Ministério da Saúde,3
8,20000,Presidência da República,2
9,39000,"Ministério dos Transportes, Portos e Aviação",2


# 3. Consultas

Expresse cada consulta abaixo utilizando SQL: 


**1.** Retorne a quantidade de palavras em cada registro da coluna nome_orgao da tabela orgão.

In [9]:
df = pd.read_sql_query("SELECT NOME_ORGAO, (LENGTH(TRIM(NOME_ORGAO)) - " +
                       "LENGTH(replace(NOME_ORGAO, ' ', '')) + 1) AS WORDS " +
                       "FROM ORGAO ", conn)
df

,NOME_ORGAO,WORDS
0,Presidência da República,3
1,"Ministério da Agricultura, Pecuária e Abastec",6
2,"Ministério da Ciência, Tecnologia, Inovações",5
3,Ministério da Fazenda,3
4,Ministério da Educação,3
5,Ministério da Justiça e Segurança Pública,6
6,Ministério das Relações Exteriores,4
7,Ministério da Saúde,3
8,"Ministério dos Transportes, Portos e Aviação",6
9,Ministério do Meio Ambiente,4


**2.** Dado a coluna nome_orgao, retorne a primeira posição que a string 'da' aparece da tabela orgão 

In [10]:
df = pd.read_sql_query("SELECT * " +
                       "FROM ORGAO " +
                       "WHERE NOME_ORGAO LIKE '%da%' " +
                       "LIMIT 1", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,20000,Presidência da República,2


**3.** Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [11]:
df = pd.read_sql_query("SELECT O.NOME_ORGAO, SUM(D.VALOR_DESPESA) AS DESPESAS_TOTAIS " +
                       "FROM ORGAO AS O, DESPESA_PUBLICA AS D " +
                       "WHERE O.COD_ORGAO = D.COD_ORGAO AND D.ANO_DESPESA = 2018 " +
                       "GROUP BY O.NOME_ORGAO "
                       "ORDER BY DESPESAS_TOTAIS DESC", conn)
df

,NOME_ORGAO,DESPESAS_TOTAIS
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,"Ministério da Ciência, Tecnologia, Inovações",0.00
6,Ministério da Fazenda,0.00
7,Ministério da Integração Nacional,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


**4.** Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.



In [12]:
df = pd.read_sql_query("SELECT D.MES_DESPESA AS MES, D.ANO_DESPESA AS ANO, " +
                       "D.MODALIDADE_DESPESA, P.NOME_PROGRAMA_GOVERNO AS PROGRAMA, " +
                       "D.VALOR_DESPESA AS DESPESA " +
                       "FROM DESPESA_PUBLICA AS D, ORGAO AS O, PROGRAMA_GOVERNO AS P " +
                       "WHERE D.COD_ORGAO = O.COD_ORGAO AND " +
                       "D.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO AND " +
                       "D.ANO_DESPESA = 2018 AND D.MES_DESPESA <= 6 " +
                       "AND O.NOME_ORGAO = 'Ministério da Fazenda' " +
                       "ORDER BY D.MES_DESPESA ASC", conn)
df

,MES,ANO,MODALIDADE_DESPESA,PROGRAMA,DESPESA
0,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
1,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
3,4,2018,Reserva de Contingência,GESTAO AEROPORTUARIA,0
4,4,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
5,5,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
6,6,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


**5.** Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [13]:
df = pd.read_sql_query("SELECT COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO " +
                       "FROM PROGRAMA_GOVERNO " +
                       "WHERE COD_PROGRAMA_GOVERNO IN " +
                       "(SELECT COD_PROGRAMA_GOVERNO " +
                       "FROM DESPESA_PUBLICA " +
                       "WHERE ANO_DESPESA = 2018 AND COD_DETALHAMENTO IN " +
                       "(SELECT DF.COD_DETALHAMENTO " +
                       "FROM DETALHAMENTO_FUNCAO AS DF, FUNCAO AS F " +
                       "WHERE DF.COD_FUNCAO = F.COD_FUNCAO AND F.NOME_FUNCAO = 'Educação') " +
                       "GROUP BY COD_PROGRAMA_GOVERNO " +
                       "ORDER BY SUM(VALOR_DESPESA) DESC " +
                       "LIMIT 3)", conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


**6.** Transforme os caracteres em minúsculo e remova todos os termos 'da' na coluna nome_orgao da tabela orgão.

In [14]:
df = pd.read_sql_query("SELECT REPLACE(LOWER(NOME_ORGAO), 'da', '') AS [NOME DO ORGAO] " +
                       "FROM ORGAO", conn)
df

,NOME DO ORGAO
0,presidência república
1,"ministério agricultura, pecuária e abastec"
2,"ministério ciência, tecnologia, inovações"
3,ministério fazen
4,ministério educação
5,ministério justiça e segurança pública
6,ministério s relações exteriores
7,ministério saúde
8,"ministério dos transportes, portos e aviação"
9,ministério do meio ambiente


**7.** Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

In [15]:
df = pd.read_sql_query("SELECT NOME_ORGAO, COUNT(DISTINCT COD_PROGRAMA_GOVERNO) " +
                       "AS [QUANTIDADE DE PROGRAMAS]"
                       "FROM ORGAO AS O, DESPESA_PUBLICA AS D " +
                       "WHERE O.COD_ORGAO = D.COD_ORGAO AND D.VALOR_DESPESA > 0 " +
                       "GROUP BY D.COD_ORGAO", conn)
df

,NOME_ORGAO,QUANTIDADE DE PROGRAMAS
0,"Ministério da Agricultura, Pecuária e Abastec",2
1,Ministério da Educação,4
2,Ministério da Saúde,2
3,Ministério da Defesa,2
4,Ministério do Desenvolvimento Social,1


**8.** Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [16]:
df = pd.read_sql_query("SELECT P.NOME_PROGRAMA_GOVERNO, S.NOME_SUBFUNCAO, " +
                       "COUNT(D.COD_DESPESA), SUM(VALOR_DESPESA) " +
                       "FROM DESPESA_PUBLICA AS D, PROGRAMA_GOVERNO AS P, " +
                       "DETALHAMENTO_FUNCAO AS DF, SUBFUNCAO AS S " +
                       "WHERE D.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO " +
                       "AND D.COD_DETALHAMENTO = DF.COD_DETALHAMENTO " +
                       "AND DF.COD_SUBFUNCAO = S.COD_SUBFUNCAO " +
                       "GROUP BY NOME_SUBFUNCAO", conn)
df

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,COUNT(D.COD_DESPESA),SUM(VALOR_DESPESA)
0,MAIS MEDICOS,Administração geral,31,0.00
1,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
2,VIVER SEM LIMITE,Assistência ao portador de deficiência,5,0.00
3,BRASIL SEM MISERIA,Assistência comunitária,148,227883.77
4,"CRACK, E POSSIVEL VENCER",Assistência hospitalar e ambulatorial,191,40181.19
5,MAIS MEDICOS,Atenção básica,78,12000.00
6,BRASIL SEM MISERIA,Comunicação social,2,0.00
7,CADEIAS PUBLICAS,Custódia e reintegração social,9,0.00
8,DESASTRES NATURAIS,Defesa civil,6,0.00
9,OLIMPIADAS,Defesa terrestre,8,0.00


**9.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

In [17]:
df = pd.read_sql_query("SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, " +
                       "MODALIDADE_DESPESA, VALOR_DESPESA " +
                       "FROM DESPESA_PUBLICA " +
                       "WHERE COD_ORGAO IN (SELECT COD_ORGAO " +
                       "FROM ORGAO " +
                       "ORDER BY QTD_ORGAOS_SUBORDINADOS DESC " +
                       "LIMIT 1)", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,196,10,2018,Reserva de Contingência,0.0
1,197,1,2018,Reserva de Contingência,0.0
2,198,2,2018,Reserva de Contingência,0.0
3,199,3,2018,Reserva de Contingência,0.0
4,200,4,2018,Reserva de Contingência,0.0
...,...,...,...,...,...
289,485,8,2018,Reserva de Contingência,2400.0
290,486,9,2018,Reserva de Contingência,0.0
291,487,10,2018,Reserva de Contingência,0.0
292,488,11,2018,Reserva de Contingência,0.0


**10.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior valor total em despesas durante o ano de 2018.

In [18]:
df = pd.read_sql_query("SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, " +
                       "MODALIDADE_DESPESA, VALOR_DESPESA " +
                       "FROM DESPESA_PUBLICA " +
                       "WHERE COD_ORGAO IN (SELECT COD_ORGAO " +
                       "FROM DESPESA_PUBLICA " +
                       "WHERE ANO_DESPESA = 2018 " +
                       "GROUP BY COD_ORGAO " +
                       "ORDER BY SUM(VALOR_DESPESA) DESC) " +
                       "LIMIT 1", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,0,11,2018,Reserva de Contingência,0


**11.** Liste os códigos e nomes dos programas de governo relacionados ao órgão (ou órgãos, caso haja empate) que teve menos registros de despesas públicas durante o ano.

In [19]:
df = pd.read_sql_query("SELECT P.COD_PROGRAMA_GOVERNO, P.NOME_PROGRAMA_GOVERNO " +
                       "FROM  PROGRAMA_GOVERNO AS P, DESPESA_PUBLICA AS D " +
                       "WHERE D.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO " +
                       "GROUP BY D.COD_ORGAO " +
                       "HAVING COUNT(D.COD_DESPESA) IN (SELECT MIN(REGISTROS_DESPESAS) " +
                       "FROM (SELECT DP.COD_PROGRAMA_GOVERNO, " +
                       "COUNT(DP.COD_DESPESA) AS REGISTROS_DESPESAS " +
                       "FROM DESPESA_PUBLICA AS DP " +
                       "GROUP BY DP.COD_ORGAO))", conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,19,OLIMPIADAS
1,24,BRASIL SEM MISERIA


**12.**  Álgebra relacional: projeção do NOME_FUNCAO e NOME_SUBFUNCAO da junção natural das tabelas FUNCAO, DETALHAMENTO_FUNCAO e SUBFUNCAO

$$
RESULT ← π_{NOME\_FUNCAO,\ NOME\_SUBFUNCAO}((FUNCAO ⨝ DETALHAMENTO\_FUNCAO) ⨝ SUBFUNCAO)
$$

In [20]:
df = pd.read_sql_query("SELECT NOME_FUNCAO, NOME_SUBFUNCAO " +
                       "FROM (FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO " +
                       "NATURAL JOIN SUBFUNCAO)", conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


**13.**  Álgebra relacional: projeção de MODALIDADE_DESPESA, VALOR_DESPESA e NOME_PROGRAMA_GOVERNO da junção natural das tabelas PROGRAMA_GOVERNO e DESPESA_PUBLICA onde a MODALIDADE_DESPESA = 'Reserva de Contingência'

$$
RES\_CONT ← σ_{MODALIDADE\_DESPESA\ =\ 'Reserva\ de\ Contingência'}(PROGRAMA\_GOVERNO ⨝ DESPESA\_PUBLICA)
$$
$$
RESULT ← π_{MODALIDADE\_DESPESA, \ VALOR\_DESPESA, \ NOME\_PROGRAMA\_GOVERNO}(RES\_CONT)
$$

In [21]:
df = pd.read_sql_query("SELECT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO " +
                       "FROM (PROGRAMA_GOVERNO NATURAL JOIN DESPESA_PUBLICA) " +
                       "WHERE MODALIDADE_DESPESA = 'Reserva de Contingência'", conn)
df

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
...,...,...,...
671,Reserva de Contingência,0.0,BRASIL SEM MISERIA
672,Reserva de Contingência,0.0,BRASIL SEM MISERIA
673,Reserva de Contingência,0.0,BRASIL SEM MISERIA
674,Reserva de Contingência,0.0,BRASIL SEM MISERIA


**14.**  Álgebra relacional: projeção de VALOR_DESPESA e NOME_ORGAO da junção natural das tabelas DESPESA_PUBLICA e ORGAO onde o NOME_ORGAO = 'Ministério da Fazenda'

$$
MIN\_FAZ ← σ_{NOME\_ORGAO\ =\ 'Ministério\ da\ Fazenda'}(DESPESA\_PUBLICA ⨝ ORGAO)
$$
$$
RESULT ← π_{VALOR\_DESPESA, \ NOME\_ORGAO}(MIN\_FAZ)
$$

In [22]:
df = pd.read_sql_query("SELECT VALOR_DESPESA, NOME_ORGAO " +
                       "FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO) " +
                       "WHERE NOME_ORGAO = 'Ministério da Fazenda'", conn)
df

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


**15.**  Álgebra relacional: projeção de NOME_PROGRAMA_GOVERNO e VALOR_DESPESA da junção natural das tabelas DESPESA_PUBLICA, PROGRAMA_GOVERNO e ORGAO onde o NOME_ORGAO = 'Ministério da Saúde'

$$
MIN\_SAU ← σ_{NOME\_ORGAO\ =\ 'Ministério\ da\ Saúde'}((DESPESA\_PUBLICA ⨝ PROGRAMA\_GOVERNO) ⨝ ORGAO)
$$
$$
RESULT ← \pi_{NOME\_PROGRAMA\_GOVERNO,\ VALOR\_DESPESA}(MIN\_SAU)
$$

In [23]:
df = pd.read_sql_query("SELECT NOME_PROGRAMA_GOVERNO, VALOR_DESPESA " +
                       "FROM (DESPESA_PUBLICA NATURAL JOIN PROGRAMA_GOVERNO NATURAL JOIN ORGAO) " +
                       "WHERE NOME_ORGAO = 'Ministério da Saúde'", conn)
df

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
...,...,...
214,MAIS MEDICOS,0.0
215,MAIS MEDICOS,0.0
216,MAIS MEDICOS,0.0
217,MAIS MEDICOS,0.0


# 4. Fechamento da conexão com o banco

* Após serem realizadas todas as consultas necessárias, é aconselhavel encerrar formalmente a conexão com o banco de dados

In [24]:
conn.close()